In [ ]:
#|hide
#|default_exp core

# core

> Fast scripts using daemon mode

In [ ]:
#|export
import socket
from contextlib import redirect_stderr, redirect_stdout
from io import StringIO
from multiprocessing import get_context
from socketserver import TCPServer, StreamRequestHandler

from fastcore.meta import *
from fastcore.net import *
from fastcore.script import *
from fastcore.utils import *

In [ ]:
#|hide
from nbprocess.showdoc import *

In [ ]:
import time
from fastcore.test import *

In [ ]:
#|export
import struct
from io import BytesIO

In [ ]:
#|export
def pack_streams(streams): # TODO: use chunked?
    "Pack a list of variable length utf-8 strings"
    streams = [o.encode('utf-8') if isinstance(o,str) else o for o in streams]
    ls = [len(o) for o in streams]
    fmt = '!'+''.join(f'L{l}s' for l,s in zip(ls,streams))
    vs = sum(list(zip(ls, streams)), ())
    return struct.pack(fmt, *vs)

In [ ]:
streams = ['from fastcore.all import *\n', 'nbprocess_clean --stdin']
s = pack_streams(streams); s

b'\x00\x00\x00\x1bfrom fastcore.all import *\n\x00\x00\x00\x17nbprocess_clean --stdin'

In [ ]:
#|export
def send_streams(w, streams): w(pack_streams(streams)) # TODO: needed?

In [ ]:
#|export
def readlen(r):
    "Read the length of the next parameter"
    return struct.unpack('!L', r(4))[0]

In [ ]:
readlen(BytesIO(s).read)

27

In [ ]:
#|export
def recv_streams(r):
    "Receive a 2-tuple of variable length utf-8 strings"
    res = []
    for _ in range(2):
        lv = readlen(r)
        v = struct.unpack(f'{lv}s', r(lv))[0].decode('utf-8')
        res.append(v)
    return res

In [ ]:
recv_streams(BytesIO(s).read)

['from fastcore.all import *\n', 'nbprocess_clean --stdin']

In [ ]:
#|export
def send_recv(streams, port, host=None, dgram=False):
    "Wraps `start_client`, `send_streams`, and `recv_streams`"
    with start_client(port, host=host, dgram=dgram) as client:
        with client.makefile('wb') as f: send_streams(f.write, streams)
        with client.makefile('rb') as f: return recv_streams(f.read)

In [ ]:
#|export
def _handle(cmd, stdin, args):
    "Execute `cmd` with `stdin` and `args`, and return `stdout`"
    sys.argv = [cmd.__name__] + (args.split(' ') if args else [])
    sys.stdin,sys_stdin = StringIO(stdin),sys.stdin
    with redirect_stdout(StringIO()) as stdout, redirect_stderr(StringIO()) as stderr: cmd()
    sys.stdin = sys_stdin
    return tuple(o.getvalue().encode() for o in (stdout,stderr))

In [ ]:
#|export
class DaemonHandler(StreamRequestHandler):
    "Execute server's `cmd` with request args using server's process pool"
    def handle(self):
        stdin,args = recv_streams(self.rfile.read)
        print(f'{stdin=} {args=}')
        future = self.server.pool.submit(_handle, self.server.cmd, stdin, args)
        stdout, stderr = future.result()
        print(f'{stdout=} {stderr=}')
        send_streams(self.wfile.write, (stdout,stderr))

`DaemonHandler`'s primary use-case is in `DaemonServer`, however it supports any `socketserver.BaseServer` that has `pool` and `cmd` attributes.

In [ ]:
#|export
class DaemonServer(TCPServer): # TODO: could be a mixin to support other servers; `Pool(ed)Server`?
    "A `TCPServer` that executes `cmd` with request args using a process pool"
    @delegates(TCPServer)
    def __init__(self, server_address, cmd, RequestHandlerClass=DaemonHandler, timeout=None, **kwargs):
        self.cmd = cmd # TODO: is this the best place for `cmd`?
        if timeout is not None: self.timeout = timeout
        self.allow_reuse_address = True
        super().__init__(server_address, RequestHandlerClass)
        
    def server_activate(self):
        self.pool = ProcessPoolExecutor(mp_context=get_context('fork')) # TODO: make ctx configurable?
        super().server_activate()
        
    def server_close(self):
        if hasattr(self,'pool'): self.pool.shutdown()
        super().server_close()
        
    def handle_timeout(self): return True

Here's an example of how to use `DaemonServer`. First, define the `cmd`. It should have no arguments itself but rather parse its arguments from `sys.argv`. Its return value isn't used, instead it should write to `stdout`.

In [ ]:
def _cmd():
    import sys
    name = sys.argv[1]
    print(f"Hello, {name}! stdin here is: '{sys.stdin.getvalue()}'")
    sys.stderr.write('Error!')

Then start the server. We start it with `handle_request` in a separate thread:

In [ ]:
addr = host,port = 'localhost',9999
@threaded
def _f():
    with DaemonServer(addr, _cmd) as srv: srv.handle_request()
_f()
time.sleep(0.2) # wait for server to start

...so that we can send a request and print its response:

In [ ]:
send_recv(('Input to stdin', 'world'), port, host)

stdin='Input to stdin' args='world'
stdout=b"Hello, world! stdin here is: 'Input to stdin'\n" stderr=b'Error!'


["Hello, world! stdin here is: 'Input to stdin'\n", 'Error!']

In [ ]:
#|export
def fastdaemon_serve(cmd, port, host=None, timeout=None):
    "Serve `cmd` on `port`, with optional `host` and `timeout`"
    host = host or socket.gethostname()
    with DaemonServer((host,port), cmd, timeout=timeout) as srv:
        while not srv.handle_request(): pass

A convenient wrapper to instantiate and start a `DaemonServer` that handles requests until it's interrupted or times out. Here's the previous example using `fastdaemon_serve`:

In [ ]:
@threaded
def _f(): fastdaemon_serve(_cmd, port, timeout=2)
_f()
time.sleep(0.2) # wait for server to start

In [ ]:
send_recv(('', 'world'), port, host)

stdin='' args='world'
stdout=b"Hello, world! stdin here is: ''\n" stderr=b'Error!'


["Hello, world! stdin here is: ''\n", 'Error!']

In [ ]:
#|export
from nbprocess.clean import wrapio

In [ ]:
#|export
@call_parse(nested=True)
def fastdaemon_client(
    port:int, # Port to connect to
    host:str=None): # Host to connect to
    "Forward `sys.args` and `sys.stdin` to `fastdaemon_server` and write response `stdout` and `stderr`"
    args = ' '.join(sys.argv[1:])
    stdin = wrapio(sys.stdin).read() if not sys.stdin.isatty() else '' # TODO: wrapio needed?
    stdout,stderr = send_recv((stdin,args), port, host)
    sys.stderr.write(stderr)
    sys.stdout.write(stdout)

Example usage:

```sh
python client.py 9999 -- arg --keyword value
```

where `client.py` contains:

```python
from fastdaemon.core import fastdaemon_client
fastdaemon_client()
```

## Export -

In [ ]:
#|hide
#|eval: false
from nbprocess.doclinks import nbprocess_export
nbprocess_export()